In [2]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

from src.features import build_features

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [8]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO

In [9]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

In [10]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "sobre_75": "Edad > 75", # Sobre 75 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.

In [11]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR, query_strings, COLUMNAS_POBLACION
)
# Une todos los resultados en un unico DataFrame
poblaciones_estratos_calculados = pd.concat(poblaciones_estratos_calculados)
poblaciones_estratos_calculados = poblaciones_estratos_calculados.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

---

En este apartado se leerán las incidencias, junto a las poblaciones respectivas donde se tienen que aplicar tales incidencias. Posteriormente, se calcularán los casos teóricos para cada problema de salud, y para cada estrato definido.

## Lectura de Incidencias por problema de salud

In [16]:
incidencias["Casos (Cada 100.000)"]

0                13.3
1                21.5
2                  40
3                  40
4                  40
5                 1.2
6                  40
7                  40
8                  40
9                11.1
10               7.63
11            Separar
12            Separar
13            Separar
14              0.843
15                2.5
16               7.63
17    Hay Prevalencia
18            Separar
19                550
20               0.57
21                200
22                 80
23               3.05
24                  8
25                  8
26                535
27               1.97
28               93.7
29               28.8
30              0.225
31                365
32               1.45
33                  -
34    Hay Prevalencia
35                 56
36              8.825
37              10750
38                500
39                500
Name: Casos (Cada 100.000), dtype: object

In [33]:
RUTA_PLANILLA_INCIDENCIAS = "../data/external/incidencias_y_prevalencias.xlsx"
COLUMNAS_A_UTILIZAR = ["Diagnostico", "Estadística", "Casos (Cada 100.000)", "Edad Incidencia"]


# Lee planilla de incidencias
incidencias = pd.read_excel(RUTA_PLANILLA_INCIDENCIAS, usecols=COLUMNAS_A_UTILIZAR)
# Convierte las incidencia a numeros e imputa las faltantes como NaN
incidencias["Casos (Cada 100.000)"] = pd.to_numeric(
    incidencias["Casos (Cada 100.000)"], errors="coerce"
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Casos (Cada 100.000)"] / 100000

In [35]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias = incidencias.merge(
    poblaciones_estratos_calculados, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias[COLUMNAS_POBLACION] = union_poblaciones_e_incidencias[
    COLUMNAS_POBLACION
].multiply(union_poblaciones_e_incidencias["rate_incidencia"], axis="index")

# Elimina rate de incidencia para claridad
union_poblaciones_e_incidencias = union_poblaciones_e_incidencias.drop(columns="rate_incidencia")

In [36]:
with pd.ExcelWriter("../data/interim/casos_teoricos_diagnosticos.xlsx") as writer:
    union_poblaciones_e_incidencias.to_excel(writer, sheet_name="casos_teoricos", index=False)
    poblaciones_estratos_calculados.to_excel(writer, sheet_name="poblacion_INE", index=False)